In [1]:
##Hoping to remove reliance on ArcGIS for ground cover processing

# This tool will not ensure that pixels and boundaries align
# That is the job of the user
import pandas
import numpy
from osgeo import gdal, gdalconst
from osgeo import ogr
import glob
import os
#import datetime
#import calendar
##import matplotlib.pyplot as plt

#Path to shapefile to use for clip/mask
pathsToShapefile = ['P:/projects/Reefscape/SRTM_H_1SEC_30M_REGIONAL/CY/cy_footprint_buffer5km.shp','P:/projects/Reefscape/SRTM_H_1SEC_30M_REGIONAL/WT/wt_footprint_buffer5km.shp'
                    ,'P:/projects/Reefscape/SRTM_H_1SEC_30M_REGIONAL/BU/bu_footprint_buffer5km.shp','P:/projects/Reefscape/SRTM_H_1SEC_30M_REGIONAL/MW/mw_footprint_buff5k.shp'
                    ,'P:/projects/Reefscape/SRTM_H_1SEC_30M_REGIONAL/FI/fi_footprint_buffer5km.shp','P:/projects/Reefscape/SRTM_H_1SEC_30M_REGIONAL/BM/bm_footprint_buff5k.shp']

#pathsToShapefile = ['P:/projects/Reefscape/SRTM_H_1SEC_30M_REGIONAL/MW/mw_footprint_buff5k.shp']

#specify individually later
#pathsToRasters = ['D:/RockFactor/raster/gbr_rockfact/hdr.adf']

outputDir = r'D:/RockFactor/raster/regional'

#Final NoData Val: can't use 0, as that is a 'valid' cover val....
ourNoData = -1


In [2]:
#define a function that will interpret a raster extent matched to the shapefil bounding box
def shapeExtent_To_Raster_Box(shapeExtent, rasterGeoInfo):
    
    #Do a bunch of stuff
    originX = rasterGeoInfo[0]
    originY = rasterGeoInfo[3]
    
    pixelWidth = rasterGeoInfo[1]
    pixelHieght = rasterGeoInfo[5]
    
    clipXMin = int((shapeExtent[0] - originX) / pixelWidth)
    clipXMax = int((shapeExtent[1] - originX) / pixelWidth) + 1
    
    clipYMin = int((shapeExtent[3] - originY) / pixelHieght)
    clipYMax = int((shapeExtent[2] - originY) / pixelHieght) + 1
    
    colCount = clipXMax - clipXMin
    rowCount = clipYMax - clipYMin
    
    newLeftCoord = originX + (clipXMin * pixelWidth)
    newTopCoord = originY + (clipYMin * pixelHieght)
    
    #This is not a Geotrans object, because I also want to return the row & col count
    #and the explicit row & col numbers to be used to clip the source raster
    newGeoInfo = (newLeftCoord, newTopCoord, pixelWidth, pixelHieght, colCount, rowCount, clipXMin, clipXMax, clipYMin, clipYMax)
    
    return newGeoInfo

In [3]:
def processRasterFile(fullFileName, whatIsIt, shapeFiles):
    
    print("Opening raster: " + os.path.basename(fullFileName) + " which represents: " + whatIsIt)
    baseRaster = gdal.Open(fullFileName)
    baseGeoInfo = baseRaster.GetGeoTransform()
    
    sourceRast = baseRaster.GetRasterBand(1).ReadAsArray()
    
    print("Base GeoInfo: " + str(baseGeoInfo))
    
    
    for thisShapefile in shapeFiles:
        
        fName = os.path.basename(thisShapefile)
        clipPolys = ogr.Open(thisShapefile)
        lyr = clipPolys.GetLayer(0)
        clipExt = lyr.GetExtent()
        
        regCode = fName[0:2]
        
        print("Extent of " + regCode + " is : " + str(clipExt))
        
        newRasterInfo = shapeExtent_To_Raster_Box(clipExt, baseGeoInfo)
        
        #clip = rast[ulY:lrY, ulX:lrX]
        clipSourceRast = sourceRast[newRasterInfo[8]:newRasterInfo[9],newRasterInfo[6]:newRasterInfo[7]]
        
        #re-mask
        clipSourceRast = numpy.ma.masked_values(clipSourceRast, baseRaster.GetRasterBand(1).GetNoDataValue())
        
        #Multiply to allow storage as useful integers
        #clipSourceRast = clipSourceRast * 10000
        clipSourceRast = clipSourceRast * 1
        
        #Change mask value
        clipSourceRast = clipSourceRast.filled(ourNoData)
        
        #Remask
        clipSourceRast = numpy.ma.masked_values(clipSourceRast, ourNoData)
        
        #Convert to Int, # i4 = 32-bit signed integer
        #clipSourceRast = numpy.array(clipSourceRast, dtype='i4')
        clipSourceRast = numpy.array(clipSourceRast, dtype='f')
        
        #Remask
        #clipSourceRast = numpy.ma.masked_values(clipSourceRast, baseRaster.GetRasterBand(1).GetNoDataValue())
        
        outputFile = os.path.join(outputDir, regCode + "_" + whatIsIt + ".tif")
        print("Saving: " + outputFile)
        
        #GDT_Byte
        #outDS = gdal.GetDriverByName("GTiff").Create(outputFile, newRasterInfo[4], newRasterInfo[5], 1, gdal.GDT_Int32)
        outDS = gdal.GetDriverByName("GTiff").Create(outputFile, newRasterInfo[4], newRasterInfo[5], 1, gdal.GDT_Float32)
        #outDS = gdal.GetDriverByName("AAIGrid").Create(outputFile, newRasterInfo[4], newRasterInfo[5], 1, gdal.GDT_Int16)
        outDS.SetGeoTransform((newRasterInfo[0], newRasterInfo[2], 0, newRasterInfo[1], 0, newRasterInfo[3]))
        
        outBand = outDS.GetRasterBand(1)
        outBand.WriteArray(clipSourceRast)
        
        # flush data to disk, set the NoData value and calculate stats
        outBand.FlushCache()
        outBand.SetNoDataValue(ourNoData)
        outDS.SetProjection(baseRaster.GetProjection())
        
        #Clear memory
        clipSourceRast = None
        outDS = None
        outBand = None
        
    
    sourceRast = None
    baseRaster = None



In [4]:
#for theRaster in pathsToRasters:
#    processRasterFile(theRaster, "lfact", pathsToShapefile)


processRasterFile('D:/RockFactor/raster/gbr_rockfact/hdr.adf', "rockfact", pathsToShapefile)

print("All done")


Opening raster: hdr.adf which represents: rockfact
Base GeoInfo: (1152688.63051, 30.0, 0.0, -1169500.4690071, 0.0, -30.0)
Extent of cy is : (1152698.6305100098, 1451508.6305100136, -1796520.469007101, -1169510.469007101)
Saving: D:/RockFactor/raster/regional\cy_rockfact.tif
Extent of wt is : (1347188.630510006, 1514148.6305100098, -2116290.469007099, -1756310.469007099)
Saving: D:/RockFactor/raster/regional\wt_rockfact.tif
Extent of bu is : (1270658.630510006, 1719798.630510006, -2728800.469007097, -2003360.469007101)
Saving: D:/RockFactor/raster/regional\bu_rockfact.tif
Extent of mw is : (1672868.6305100098, 1797558.6305100098, -2515470.469007101, -2252960.469007097)
Saving: D:/RockFactor/raster/regional\mw_rockfact.tif
Extent of fi is : (1453658.630510006, 1953138.6305100098, -2991630.469007101, -2390180.469007099)
Saving: D:/RockFactor/raster/regional\fi_rockfact.tif
Extent of bm is : (1797998.630509999, 2077338.6305099984, -3080010.4690071, -2725940.4690071)
Saving: D:/RockFactor/r